In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
# import our chat-bot intents file
import json
with open('data/responses.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

8 documents
2 classes ['goodbye', 'greeting']
14 unique stemmed words ['anyon', 'ar', 'bye', 'day', 'good', 'goodby', 'hello', 'hi', 'how', 'is', 'lat', 'see', 'ther', 'you']


In [5]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-5-9d149b3413ff>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [6]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
---------------------------------
Run id: Y7K47A
Log directory: tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 8
Validation samples: 0
--
Training Step: 1  | time: 0.151s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 8/8
--
Training Step: 2  | total loss: 0.62385 | time: 0.002s
| Adam | epoch: 002 | loss: 0.62385 - acc: 0.2250 -- iter: 8/8
--
Training Step: 3  | total loss: 0.68033 | time: 0.003s
| Adam | epoch: 003 | loss: 0.68033 - acc: 0.5523 -- iter: 8/8
--
Training Step: 4  | total loss: 0.68953 | time: 0.002s
| Adam | epoch: 004 | loss: 0.68953 - acc: 0.6068 -- iter: 8/8
--
Training Step: 5  | total loss: 0.69145 | time: 0.003s
| Adam | epoch: 005 | loss: 0.69145 - acc: 0.6194 -- iter: 8/8
--
Training Step: 6  | total loss: 0.69182 | time: 0.002s

UnknownError: Failed to rename: C:\Users\Daniel Krasovski\Documents\GitHub\Project_Oreo\models\model.tflearn.index.tempstate5008535237807874328 to: C:\Users\Daniel Krasovski\Documents\GitHub\Project_Oreo\models\model.tflearn.index : Access is denied.
; Input/output error
	 [[node save/SaveV2 (defined at C:\Users\Daniel Krasovski\anaconda3\envs\Project_Oreo\lib\site-packages\tflearn\helpers\trainer.py:134) ]]

Errors may have originated from an input operation.
Input Source operations connected to node save/SaveV2:
 val_loss (defined at C:\Users\Daniel Krasovski\anaconda3\envs\Project_Oreo\lib\site-packages\tflearn\helpers\trainer.py:666)	
 FullyConnected_1/b (defined at C:\Users\Daniel Krasovski\anaconda3\envs\Project_Oreo\lib\site-packages\tflearn\variables.py:57)	
 FullyConnected_2/b/Adam_1 (defined at C:\Users\Daniel Krasovski\anaconda3\envs\Project_Oreo\lib\site-packages\tflearn\helpers\trainer.py:713)	
 Global_Step (defined at C:\Users\Daniel Krasovski\anaconda3\envs\Project_Oreo

In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )